In [4]:
import cv2
import numpy as np
import os
import time
from tqdm import tqdm

def detect_features(image, algorithm):
    if algorithm == 'SIFT':
        detector = cv2.SIFT_create()
    elif algorithm == 'ORB':
        detector = cv2.ORB_create()
    elif algorithm == 'AKAZE':
        detector = cv2.AKAZE_create()
    elif algorithm == 'BRISK':
        detector = cv2.BRISK_create()
    else:
        raise ValueError(f"Unsupported algorithm: {algorithm}")

    start_time = time.time()
    keypoints, descriptors = detector.detectAndCompute(image, None)
    end_time = time.time()

    return keypoints, descriptors, end_time - start_time

def evaluate_feature_quality(image1, image2, keypoints1, keypoints2, descriptors1, descriptors2):
    # 특징점 매칭
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)

    # 매칭 품질 평가
    good_matches = [m for m in matches if m.distance < 0.7 * matches[0].distance]
    return len(good_matches) / len(keypoints1) if keypoints1 else 0

def evaluate_repeatability(image1, image2, keypoints1, keypoints2):
    # 이미지 크기 가져오기
    h, w = image1.shape[:2]

    # 키포인트를 픽셀 좌표로 변환
    pts1 = np.float32([kp.pt for kp in keypoints1]).reshape(-1, 1, 2)
    pts2 = np.float32([kp.pt for kp in keypoints2]).reshape(-1, 1, 2)

    # 최소 4개의 매칭점이 필요합니다
    if len(pts1) < 4 or len(pts2) < 4:
        return 0  # 충분한 점이 없으면 0을 반환

    try:
        # 호모그래피 계산
        H, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, 5.0)

        if H is None:
            return 0  # 호모그래피를 찾을 수 없으면 0을 반환

        # 반복성 계산
        good_pts1 = []
        good_pts2 = []
        for (x1, y1), (x2, y2), m in zip(pts1.reshape(-1, 2), pts2.reshape(-1, 2), mask):
            if m:
                pt1 = np.array([x1, y1, 1]).reshape(3, 1)
                pt2_pred = H.dot(pt1).reshape(3)
                pt2_pred = pt2_pred[:2] / pt2_pred[2]
                if np.linalg.norm(pt2_pred - [x2, y2]) < 3:  # 3픽셀 임계값
                    good_pts1.append((x1, y1))
                    good_pts2.append((x2, y2))

        return len(good_pts1) / len(pts1) if len(pts1) > 0 else 0

    except cv2.error:
        return 0  # OpenCV 오류 발생 시 0을 반환
def evaluate_algorithm_extended(algorithm, image_folder):
    image_files = sorted(os.listdir(image_folder))
    quality_scores = []
    repeatability_scores = []

    for i in tqdm(range(len(image_files) - 1), desc=f"Evaluating {algorithm}"):
        image1_path = os.path.join(image_folder, image_files[i])
        image2_path = os.path.join(image_folder, image_files[i+1])

        image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
        image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

        if image1 is None or image2 is None:
            continue

        # 특징점 검출 및 디스크립터 계산
        keypoints1, descriptors1, _ = detect_features(image1, algorithm)
        keypoints2, descriptors2, _ = detect_features(image2, algorithm)

        # 품질 평가
        quality_score = evaluate_feature_quality(image1, image2, keypoints1, keypoints2, descriptors1, descriptors2)
        quality_scores.append(quality_score)

        # 반복성 평가
        repeatability_score = evaluate_repeatability(image1, image2, keypoints1, keypoints2)
        repeatability_scores.append(repeatability_score)

    return np.mean(quality_scores), np.mean(repeatability_scores)

# 메인 실행 부분
image_folder = "/content/drive/MyDrive/KeyPoint/dataset_split/test/damaged"
algorithms = ['SIFT', 'ORB', 'AKAZE', 'BRISK']

extended_results = {}

for algorithm in algorithms:
    quality, repeatability = evaluate_algorithm_extended(algorithm, image_folder)
    extended_results[algorithm] = {'quality': quality, 'repeatability': repeatability}

print("Extended Results:")
for alg, res in extended_results.items():
    print(f"{alg}: Quality = {res['quality']:.4f}, Repeatability = {res['repeatability']:.4f}")

Evaluating BRISK: 100%|██████████| 59/59 [00:49<00:00,  1.20it/s]

Extended Results:
SIFT: Quality = 0.0000, Repeatability = 0.0000
ORB: Quality = 0.0000, Repeatability = 0.0011
AKAZE: Quality = 0.0000, Repeatability = 0.0000
BRISK: Quality = 0.0000, Repeatability = 0.0000
